In [1]:
import nltk

In [2]:
nltk.download('punkt')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\l1742\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\l1742\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
import os
from bs4 import BeautifulSoup # for parsing of the content got bby hitting a specific url
from nltk.tokenize import word_tokenize # for generating tokens from text
from nltk.corpus import stopwords # for getting list of stop-words from nltk package

from nltk.stem import PorterStemmer # for doing stemming on tokens list

import re # for finding position of every token within documents

from collections import OrderedDict
import os, sys # for filing
import math # for square root



In [5]:
def returning_document_list(path):
    documents = reading_files_to_get_text(path)
    return documents


def reading_files_to_get_text(path):
    docs= os.listdir(path)
    return docs

In [6]:
def find_positions_of_single_word(input_string, target_word):
    target_list=[]
    #Start writing your code here
    input_string=input_string.split(" ")
    pos=1
    for i in input_string:
        if i==target_word:
            target_list.append(pos)
        pos+=1


    return target_list

In [7]:
def creation_of_tokens(path, documents,doc_ID,read_html):
    
    one_document_token_list=[]
    stop_words = set(stopwords.words('english')) # getting stop-words from nltk package
    porter = PorterStemmer() # making porterStemmer class object for doing stemming
    
    html_text= read_html.text # getting text from html
    text_without_punctuation = removing_punctuation_from_string(html_text) # removing punctuation from text
    tokens_list = word_tokenize(text_without_punctuation) # tokenization of text


    new_html_text = ""
    for stemmed_html_word in tokens_list:
        new_html_text = new_html_text + " "+(porter.stem(stemmed_html_word))
    
#     print(doc_ID, new_html_text)    
    filtered_tokens = removing_stop_words(tokens_list,stop_words) # removing stop words from token list
    
    # for stemming
    portered_word_list = []
    for word in filtered_tokens:
        portered_word_list.append(porter.stem(word))


    for word in portered_word_list:
        position = find_positions_of_single_word(new_html_text, word)
#         position = [p for p in range(len(new_html_text)) if new_html_text.startswith(word, p)]
#         position = [p.start() for p in re.finditer(word, html_text)]
        term_frequency = len(position)
        one_document_token_list.append( (word, doc_ID, term_frequency,position ) )
                
    # to remove duplicates due to position list________
    temp_list = []
    for tuplee in one_document_token_list:
        position_list_lenght= len(tuplee[3])
        if (tuplee not in temp_list) and (position_list_lenght>0):
            temp_list.append(tuplee)

    one_document_token_list = temp_list
#         #__________________________________________________

    return one_document_token_list
    



def removing_punctuation_from_string(text_string):
    
    text_string= text_string.lower().replace("|"," ").replace("!"," ")
    # define punctuation
    
    text_string = text_string.replace("("," ").replace(")"," ").replace("-"," ").replace("["," ").replace("]"," ").replace("{"," ").replace("}"," ")
    text_string = text_string.replace(";"," ").replace(":"," ").replace("'"," ").replace("/"," ").replace(","," ").replace("<"," ").replace(">"," ")
    text_string = text_string.replace("."," ").replace("?"," ").replace("@"," ").replace("#"," ").replace("$"," ").replace("%"," ").replace("^"," ")
    text_string = text_string.replace("*"," ").replace("_"," ").replace("~"," ").replace("&"," ").replace("—"," ").replace("»"," ")
    text_string = text_string.replace("–"," ").replace(('‘')," ").replace("'’'"," ").replace("â"," ").replace("+"," ")
    text_string = text_string.replace("'©'"," ")
    
    
    punctuations = '\'"~'
    # remove punctuation from the string
    no_punct = ""
    for char in text_string:
        if char not in punctuations:
            no_punct = no_punct + char
    return no_punct


def removing_stop_words(tokens_List, stop_words):
    filtered_Tokens = [] 
    for word in tokens_List: # for removing stop words from tokens
        if word not in stop_words: 
            filtered_Tokens.append(word)
    return filtered_Tokens
            

In [8]:
def generating_gap_list_for_position_list(positon_list):
    positon_list_lenght = (len(positon_list))
    keyy_list_index = 0
    previous_position = 0
    temp_posting_list = []
    for itemm in positon_list:
        if(keyy_list_index > 0):
            new_previous_position = itemm
            temp_posting_list.append((itemm - previous_position ))
            previous_position = new_previous_position
        
        elif (keyy_list_index == 0):
                temp_posting_list.append(itemm)
                previous_position = itemm
        keyy_list_index = keyy_list_index + 1
    
    return temp_posting_list
            



def generating_gap_list_for_docID(sorted_dictionary):
    temp_dictionary = {}
    
    for keyy in sorted_dictionary: # accessing dictionary key wise
        keyy_list_length = (len(sorted_dictionary[keyy]))
        keyy_list_index = 0
        previous_doc_id = 0
        posting= temp_dictionary.get(keyy, None)
        for keyy_list in sorted_dictionary[keyy]: # accessing dictionary main list's elements tuple wise and document frequency 
            if(  keyy_list_index != ( keyy_list_length -1)): # to spearate document frequency present at last position in dictionary main list's elements
                if( keyy_list_index > 0  ): # for separating first document id 
                    
                    new_previous_doc_id = keyy_list[1]
                    keyy_list_doc_id = keyy_list[1] - previous_doc_id # n_doc_id = n_doc_id - n-1_doc_id
                    key_postion_list = generating_gap_list_for_position_list(keyy_list[3]) # for generating gaps in positon list
                    
                    posting.append( (keyy_list[0], keyy_list_doc_id, keyy_list[2], key_postion_list  ) )
                    previous_doc_id = new_previous_doc_id # setting previous doc id for next iteration of loop
                
                else: # for accessing first document id 
                    
                    previous_doc_id = keyy_list[1] # setting previous doc id for next iteration of loop
                    key_postion_list = generating_gap_list_for_position_list(keyy_list[3]) # for generating gaps in positon list
                    posting = [(keyy_list[0], keyy_list[1], keyy_list[2],key_postion_list)]
            
            elif(  keyy_list_index == ( keyy_list_length -1) ): # for accessing first document frequency 
                posting.append(keyy_list)        # appending documenting frequency in the list
            keyy_list_index = keyy_list_index + 1
        temp_dictionary[keyy] = posting
        
    return temp_dictionary


In [9]:
def witing_to_files(write_dict,n_call):
    string_to_write_in_posting_list=""
    string_to_write_in_index_terms=""
    starting_bytes = 0
    posting_file_name = "index_"+str(n_call)+"_postings.txt"
    terms_file_name = "index_"+str(n_call)+"_terms.txt"
    fd1 = os.open(posting_file_name,os.O_RDWR|os.O_CREAT)
    fd2 = os.open(terms_file_name,os.O_RDWR|os.O_CREAT)
    for keyy in write_dict:
        string_to_write_in_index_terms = string_to_write_in_index_terms + str(keyy)+","
        one_posting_list= write_dict[keyy]
        keyy_list_length = (len(write_dict[keyy]))
        string_to_write_in_posting_list = string_to_write_in_posting_list + str(one_posting_list [ (keyy_list_length - 1 )]) # for adding document frequency
        keyy_list_index = 0
        
        for keyy_list in write_dict[keyy]:
            if(  keyy_list_index != ( keyy_list_length -1)): # to spearate document frequency present at last position in dictionary main list's elements
                string_to_write_in_posting_list = string_to_write_in_posting_list +"," +str(keyy_list[1])+","+str(keyy_list[2]) # for adding doc id and term frequency
                for position_no in keyy_list[3]:
                    string_to_write_in_posting_list = string_to_write_in_posting_list+"," + str(position_no)
#             elif(  keyy_list_index == ( keyy_list_length -1) ): # for accessing first document frequency 
#                 posting.append(keyy_list)

            keyy_list_index = keyy_list_index + 1
        
        line = str.encode((string_to_write_in_posting_list+"\n"))
        no_of_bytes_written = os.write(fd1, line)
        
        string_to_write_in_index_terms = string_to_write_in_index_terms + str(starting_bytes) +","+ str(no_of_bytes_written)+"\n"
        second_line = str.encode(string_to_write_in_index_terms)
        no_of_bytes_written_second = os.write(fd2, second_line)
        
        string_to_write_in_posting_list=""
        string_to_write_in_index_terms=""
        starting_bytes = starting_bytes + no_of_bytes_written
    
    os.close(fd1)
    os.close(fd2)
        

In [10]:
def returning_built_sorted_dict_for_one_directory(doc_ID,N_path,documents,n_call):
    one_document_token_list = []
    tokendict = {   # for keeping the track of tokens along with doc_ID
    }
    directory_and_docName_mapping_to_doc_id ={}
    mapped_directory_and_docName_to_doc_id_list = directory_and_docName_mapping_to_doc_id.get(N_path,None)
    n_doc_iD= doc_ID + 15


    for doc in documents: 
        dpath=(os.path.join(N_path, doc)) # adding file name at the end of file directory path
        
        opened_file = open(dpath , errors="ignore")
        read_opened_file = opened_file.read()  # reading the opened file
        
        soup = BeautifulSoup(read_opened_file,  "html.parser")
        read_html = soup.find('html') # reading html
        if(read_html != None  and doc_ID < n_doc_iD):

            doc_ID=doc_ID+1

            # for finding lenght and magnitude of document__________________
            html_N_tExt= read_html.text # getting text from html
            text_WiThout_punctuation = removing_punctuation_from_string(html_N_tExt) # removing punctuation from text
            lenght_of_document = (len(text_WiThout_punctuation))# lenght equal to number of characters in document.
            tokens_LiSt = word_tokenize(text_WiThout_punctuation) # tokenization of text
           
            magnitude_of_document = 0
            
            for Tokken in tokens_LiSt:
                No_of_instanse_of_token = (len(find_positions_of_single_word(text_WiThout_punctuation, Tokken)))
                No_of_instanse_of_token = (No_of_instanse_of_token * No_of_instanse_of_token )
                magnitude_of_document = magnitude_of_document + No_of_instanse_of_token
            
            magnitude_of_document = (math.sqrt(magnitude_of_document )) # magintude = square root of no of words in doc.
            #Ending finding lenght of document__________________
    
            if mapped_directory_and_docName_to_doc_id_list is None:
                mapped_directory_and_docName_to_doc_id_list = [[doc_ID,doc, lenght_of_document,magnitude_of_document]]
            else:
                mapped_directory_and_docName_to_doc_id_list.append([doc_ID,doc, lenght_of_document,magnitude_of_document])
            one_document_token_list = creation_of_tokens(N_path, doc, doc_ID, read_html)
        
        
            for tuplee in one_document_token_list:
                posting= tokendict.get(tuplee[0], None)
        
                if posting is None:      
                    posting= [tuplee]
                else:    
                    if tuplee not in posting: # You can also keep TF here
                        posting.append(tuplee) 
                tokendict[(tuplee[0])]= posting
        
        
        
        elif (doc_ID>=n_doc_iD):
            break
    
    sorted_dict = dict(sorted(tokendict.items()))
    directory_and_docName_mapping_to_doc_id[N_path] = mapped_directory_and_docName_to_doc_id_list

    for keyy in sorted_dict:
        sorted_dict[keyy].append(len(sorted_dict[keyy]))
        
    # print(one_document_token_list)
    print()
    print("____________________________________________________________________________________")
    print()
    print( sorted_dict)

    doc_id_gap_list_dictionary = generating_gap_list_for_docID(sorted_dict)
    print()
    print("____________________________________________________________________________________")
    print()
    print( doc_id_gap_list_dictionary)


    witing_to_files(doc_id_gap_list_dictionary,n_call) # writing to files
    
    final_list_with_relative_doc_id_and_mapping=[]
    final_list_with_relative_doc_id_and_mapping.append(doc_ID)
    final_list_with_relative_doc_id_and_mapping.append(directory_and_docName_mapping_to_doc_id)
    return final_list_with_relative_doc_id_and_mapping

In [11]:
def keeping_extra_info_in_file(extra_info_list):
    fd_3 = os.open("docInfo.txt",os.O_RDWR|os.O_CREAT)
    extra_string_info =""
    
    for extra_info in extra_info_list:       
        for keyy in extra_info:            
            for single_list_pair in extra_info[keyy]:
                extra_string_info = extra_string_info + str(single_list_pair[0])+","+str(keyy)+"/"+str(single_list_pair[1])+","+str(single_list_pair[2])+","+str(single_list_pair[3])
                line = str.encode((extra_string_info+"\n"))
                no_of_bytes_written = os.write(fd_3, line)
                extra_string_info=""
    
    os.close(fd_3)
              

In [12]:
doc_ID=0 
extra_info_list =[]

no_of_directoires = 3 
cOunteR = 0
while (cOunteR <  no_of_directoires):
    N_path =input('Enter the path:')
    documents = returning_document_list(N_path)
    finaL_list_for_onedirectory_with_mapping = returning_built_sorted_dict_for_one_directory(doc_ID,N_path,documents,(cOunteR+1))
    doc_ID = finaL_list_for_onedirectory_with_mapping[0]
    extra_info_list.append(finaL_list_for_onedirectory_with_mapping[1])
    
    
    cOunteR = cOunteR + 1




Enter the path:C:\\Users\\l1742\\OneDrive\\Desktop\\Abc\\IR\\Assignment 3\\corpus\\corpus1\\1\\

____________________________________________________________________________________

{'01': [('01', 12, 1, [90]), ('01', 13, 1, [90]), 2], '08': [('08', 12, 3, [81, 1010, 1022]), ('08', 13, 3, [81, 1010, 1022]), 2], '1': [('1', 4, 2, [100, 264]), ('1', 5, 2, [100, 264]), ('1', 8, 13, [1813, 1848, 1877, 1917, 1952, 2001, 2087, 2126, 2155, 2195, 2230, 2279, 2347]), ('1', 9, 13, [1813, 1848, 1877, 1917, 1952, 2001, 2087, 2126, 2155, 2195, 2230, 2279, 2347]), ('1', 10, 1, [2113]), ('1', 11, 1, [2113]), ('1', 12, 7, [76, 89, 579, 581, 954, 966, 991]), ('1', 13, 7, [76, 89, 579, 581, 954, 966, 991]), 8], '10': [('10', 4, 1, [597]), ('10', 5, 1, [597]), ('10', 8, 1, [1422]), ('10', 9, 1, [1422]), ('10', 10, 2, [2200, 2454]), ('10', 11, 2, [2200, 2454]), ('10', 12, 12, [476, 477, 478, 550, 551, 552, 562, 563, 564, 678, 679, 680]), ('10', 13, 12, [476, 477, 478, 550, 551, 552, 562, 563, 564, 678, 6

{'01': [('01', 12, 1, [90]), ('01', 1, 1, [90]), 2], '08': [('08', 12, 3, [81, 929, 12]), ('08', 1, 3, [81, 929, 12]), 2], '1': [('1', 4, 2, [100, 164]), ('1', 1, 2, [100, 164]), ('1', 3, 13, [1813, 35, 29, 40, 35, 49, 86, 39, 29, 40, 35, 49, 68]), ('1', 1, 13, [1813, 35, 29, 40, 35, 49, 86, 39, 29, 40, 35, 49, 68]), ('1', 1, 1, [2113]), ('1', 1, 1, [2113]), ('1', 1, 7, [76, 13, 490, 2, 373, 12, 25]), ('1', 1, 7, [76, 13, 490, 2, 373, 12, 25]), 8], '10': [('10', 4, 1, [597]), ('10', 1, 1, [597]), ('10', 3, 1, [1422]), ('10', 1, 1, [1422]), ('10', 1, 2, [2200, 254]), ('10', 1, 2, [2200, 254]), ('10', 1, 12, [476, 1, 1, 72, 1, 1, 10, 1, 1, 114, 1, 1]), ('10', 1, 12, [476, 1, 1, 72, 1, 1, 10, 1, 1, 114, 1, 1]), ('10', 1, 1, [52]), ('10', 1, 1, [52]), 10], '100': [('100', 10, 1, [2204]), ('100', 1, 1, [2204]), ('100', 1, 1, [1226]), ('100', 1, 1, [1226]), 4], '107': [('107', 6, 1, [669]), ('107', 1, 1, [669]), 2], '11': [('11', 6, 1, [47]), ('11', 1, 1, [47]), ('11', 1, 2, [1314, 288]), ('

Enter the path:C:\\Users\\l1742\\OneDrive\\Desktop\\Abc\\IR\\Assignment 3\\corpus\\corpus1\\2\\

____________________________________________________________________________________

{'0': [('0', 16, 1, [86]), ('0', 17, 1, [23]), ('0', 22, 1, [1142]), ('0', 25, 3, [1424, 1470, 1482]), ('0', 28, 1, [39]), 5], '00': [('00', 28, 4, [264, 1142, 1825, 2551]), 1], '000': [('000', 27, 2, [400, 1059]), 1], '01': [('01', 28, 4, [312, 1300, 1425, 1812]), 1], '02': [('02', 16, 1, [634]), 1], '04': [('04', 21, 5, [109, 116, 124, 130, 137]), ('04', 22, 5, [110, 117, 125, 131, 138]), 2], '06': [('06', 28, 2, [1301, 2678]), 1], '07': [('07', 21, 1, [264]), ('07', 22, 2, [265, 892]), ('07', 28, 3, [356, 449, 2507]), 3], '08': [('08', 28, 1, [314]), 1], '09': [('09', 16, 1, [633]), ('09', 28, 1, [673]), ('09', 29, 1, [790]), 3], '1': [('1', 16, 2, [84, 861]), ('1', 18, 1, [1374]), ('1', 20, 2, [34, 676]), ('1', 21, 2, [177, 808]), ('1', 22, 5, [178, 686, 1151, 1154, 1159]), ('1', 28, 9, [46, 131, 258, 


____________________________________________________________________________________

{'0': [('0', 16, 1, [86]), ('0', 1, 1, [23]), ('0', 5, 1, [1142]), ('0', 3, 3, [1424, 46, 12]), ('0', 3, 1, [39]), 5], '00': [('00', 28, 4, [264, 878, 683, 726]), 1], '000': [('000', 27, 2, [400, 659]), 1], '01': [('01', 28, 4, [312, 988, 125, 387]), 1], '02': [('02', 16, 1, [634]), 1], '04': [('04', 21, 5, [109, 7, 8, 6, 7]), ('04', 1, 5, [110, 7, 8, 6, 7]), 2], '06': [('06', 28, 2, [1301, 1377]), 1], '07': [('07', 21, 1, [264]), ('07', 1, 2, [265, 627]), ('07', 6, 3, [356, 93, 2058]), 3], '08': [('08', 28, 1, [314]), 1], '09': [('09', 16, 1, [633]), ('09', 12, 1, [673]), ('09', 1, 1, [790]), 3], '1': [('1', 16, 2, [84, 777]), ('1', 2, 1, [1374]), ('1', 2, 2, [34, 642]), ('1', 1, 2, [177, 631]), ('1', 1, 5, [178, 508, 465, 3, 5]), ('1', 6, 9, [46, 85, 127, 667, 5, 75, 811, 114, 852]), 6], '10': [('10', 16, 9, [63, 325, 242, 59, 82, 20, 20, 48, 33]), ('10', 4, 1, [755]), ('10', 1, 1, [817]), ('10', 6

Enter the path:C:\\Users\\l1742\\OneDrive\\Desktop\\Abc\\IR\\Assignment 3\\corpus\\corpus1\\3\\

____________________________________________________________________________________

{'0': [('0', 31, 2, [1855, 2117]), ('0', 32, 6, [362, 470, 576, 670, 765, 855]), ('0', 40, 1, [4]), ('0', 42, 3, [164, 196, 215]), 4], '00': [('00', 32, 1, [825]), ('00', 35, 2, [239, 603]), ('00', 41, 6, [771, 1395, 1403, 1413, 1421, 1425]), ('00', 43, 13, [84, 88, 92, 503, 505, 514, 516, 527, 529, 538, 540, 553, 555]), 4], '000': [('000', 34, 1, [255]), ('000', 37, 1, [271]), 2], '0000': [('0000', 32, 1, [298]), 1], '000744b110e2': [('000744b110e2', 33, 1, [9005]), 1], '0007b892': [('0007b892', 33, 1, [11594]), 1], '002f': [('002f', 31, 1, [10290]), 1], '0054692f': [('0054692f', 31, 1, [1139]), 1], '0055e6e6ae93': [('0055e6e6ae93', 31, 1, [12818]), 1], '005690e6': [('005690e6', 31, 1, [6267]), 1], '0072': [('0072', 31, 1, [8983]), 1], '0090': [('0090', 31, 1, [7338]), 1], '00932690c2cd': [('00932690c2cd'


____________________________________________________________________________________

{'0': [('0', 31, 2, [1855, 262]), ('0', 1, 6, [362, 108, 106, 94, 95, 90]), ('0', 8, 1, [4]), ('0', 2, 3, [164, 32, 19]), 4], '00': [('00', 32, 1, [825]), ('00', 3, 2, [239, 364]), ('00', 6, 6, [771, 624, 8, 10, 8, 4]), ('00', 2, 13, [84, 4, 4, 411, 2, 9, 2, 11, 2, 9, 2, 13, 2]), 4], '000': [('000', 34, 1, [255]), ('000', 3, 1, [271]), 2], '0000': [('0000', 32, 1, [298]), 1], '000744b110e2': [('000744b110e2', 33, 1, [9005]), 1], '0007b892': [('0007b892', 33, 1, [11594]), 1], '002f': [('002f', 31, 1, [10290]), 1], '0054692f': [('0054692f', 31, 1, [1139]), 1], '0055e6e6ae93': [('0055e6e6ae93', 31, 1, [12818]), 1], '005690e6': [('005690e6', 31, 1, [6267]), 1], '0072': [('0072', 31, 1, [8983]), 1], '0090': [('0090', 31, 1, [7338]), 1], '00932690c2cd': [('00932690c2cd', 31, 1, [13508]), 1], '0097': [('0097', 31, 1, [2613]), 1], '009e571086e0': [('009e571086e0', 33, 1, [2498]), 1], '00a100e5976b': [('00a10

In [26]:
keeping_extra_info_in_file(extra_info_list)